# Cutout with twin color pairs with 2 stars closeby in multi colors

- author Sylvie Dagoret-Campagne
- creation date 2024-06-18
- last update 2024-06-18
- affiliation : IJCLab
- Kernel **w_2024_16**

- want to see within 30 arcsec from main source thus need box size of 300 pixel

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor=None, alpha=0.1)
#props = dict(boxstyle='round')


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS


import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

import scipy
from scipy.optimize import curve_fit

In [ ]:
import gc

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time
import scipy

In [ ]:
import lsst.daf.butler as dafButler
#import lsst.summit.utils.butlerUtils as butlerUtils

In [ ]:
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import lsst.afw.table as afwTable
import lsst.afw.display.rgb as afwRgb
import lsst.afw.image as afwImage
import lsst.geom as geom

#import lsst.sphgeom
from  lsst.geom import SpherePoint, Angle
import lsst.sphgeom

In [ ]:

from astropy.wcs import WCS
from astropy.visualization import make_lupton_rgb
import gc

import lsst.afw.display as afwDisplay
from lsst.afw.image import MultibandExposure

In [ ]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
transform = AsinhStretch() + PercentileInterval(99.)

In [ ]:
# INSERT YOUR collection and tract
# for rehearsal use collection 2 which have CCDvisit
butlerRoot = "/repo/embargo"

collection1 = 'LSSTComCamSim/runs/nightlyvalidation/20240402/d_2024_03_29/DM-43612'
collection2 = 'LSSTComCamSim/runs/nightlyvalidation/20240403/d_2024_03_29/DM-43612'
collection3 = 'LSSTComCamSim/runs/nightlyvalidation/20240404/d_2024_03_29/DM-43612'
collection = 'LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240414/w_2024_15/PREOPS-5069' # COMPLETED

collectionn = collection
#collections = [collection1,collection2,collection3]
collections = [collection]
collectionStr = collectionn.replace("/", "_")
fn_ccdVisit_tracts_patches = f"ccdVisittractpatch_{collectionStr}.csv"
instrument = 'LATISS'
skymapName = "latiss_v1"
where_clause = "instrument = \'" + instrument+ "\'"
tract = 3864 # mostly for light-curves
patch_sel = 236
#tract = 5615
# tract = 5634 # interesting to view calib parameters
suptitle = collectionStr + f" inst = {instrument} tract = {tract}"

In [ ]:
#dataId = {"skymap": "latiss_v1", "tract": 5615, "instrument": "LATISS"}
dataId = {"skymap": skymapName, "tract": tract, "instrument": instrument}
repo = '/sdf/group/rubin/repo/oga/'
butler = dafButler.Butler(repo)
#t = Butler.get(table_sel, dataId=dataId, collections=collections)
registry = butler.registry

In [ ]:
skymap_auxtel ='latiss_v1'
skymap = butler.get('skyMap', skymap=skymap_auxtel, collections=collections)

In [ ]:
def remove_figure(fig):
    """
    Remove a figure to reduce memory footprint.

    Parameters
    ----------
    fig: matplotlib.figure.Figure
        Figure to be removed.

    Returns
    -------
    None
    """
    # get the axes and clear their images
    for ax in fig.get_axes():
        for im in ax.get_images():
            im.remove()
    fig.clf()       # clear the figure
    plt.close(fig)  # close the figure
    gc.collect()    # call the garbage collector

In [ ]:
def get_corners_radec(wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    corners_ra, corners_dec in decimal degrees
    """

    corners_x = [bbox.beginX, bbox.beginX, bbox.endX, bbox.endX]
    corners_y = [bbox.beginY, bbox.endY, bbox.endY, bbox.beginY]
    corners_ra = []
    corners_dec = []
    for i in range(4):
        radec = wcs.pixelToSky(corners_x[i], corners_y[i])
        corners_ra.append(radec.getRa().asDegrees())
        corners_dec.append(radec.getDec().asDegrees())
    
    return corners_ra, corners_dec

In [ ]:
def convert_fluxtomag(x) :
    """
    The object and source catalogs store only fluxes. There are hundreds of flux-related columns, 
    and to store them also as magnitudes would be redundant, and a waste of space.
    All flux units are nanojanskys. The AB Magnitudes Wikipedia page provides a concise resource 
    for users unfamiliar with AB magnitudes and jansky fluxes. To convert to AB magnitudes use:
    As demonstrated in Section 2.3.2, to add columns of magnitudes after retrieving columns of flux, users can do this:
    results_table['r_calibMag'] = -2.50 * numpy.log10(results_table['r_calibFlux']) + 31.4
    results_table['r_cModelMag'] = -2.50 * numpy.log10(results_table['r_cModelFlux']) + 31.4
    (from DP0 tutorial)
    """
    return -2.50 * np.log10(x) + 31.4

In [ ]:
def cutout_coadd(butler, ra, dec, band='r', datasetType='deepCoadd',
                 skymap=None, cutoutSideLength=51, **kwargs):
    """
    Produce a cutout from a coadd at the given ra, dec position.

    Adapted from DC2 tutorial notebook by Michael Wood-Vasey.

    Parameters
    ----------
    butler: lsst.daf.persistence.Butler
        Helper object providing access to a data repository
    ra: float
        Right ascension of the center of the cutout, in degrees
    dec: float
        Declination of the center of the cutout, in degrees
    band: string
        Filter of the image to load
    datasetType: string ['deepCoadd']
        Which type of coadd to load.  Doesn't support 'calexp'
    skymap: lsst.afw.skyMap.SkyMap [optional]
        Pass in to avoid the Butler read.  Useful if you have lots of them.
    cutoutSideLength: float [optional]
        Size of the cutout region in pixels.

    Returns
    -------
    MaskedImage
    """
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)

    if skymap is None:
        skymap = butler.get("skyMap")

    # Look up the tract, patch for the RA, Dec
    tractInfo = skymap.findTract(radec)
    patchInfo = tractInfo.findPatch(radec)
    xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    patch = tractInfo.getSequentialPatchIndex(patchInfo)

    coaddId = {'tract': tractInfo.getId(), 'patch': patch, 'band': band}
    parameters = {'bbox': bbox}

    cutout_image = butler.get(datasetType, parameters=parameters,
                              dataId=coaddId)

    return cutout_image

In [ ]:
def cutout_calexp(butler, ra, dec, visit, detector, cutoutSideLength=51, **kwargs):
    
    """
    Produce a cutout from a calexp at the given ra, dec position.

    Adapted from cutout_coadd which was adapted from a DC2 tutorial
    notebook by Michael Wood-Vasey.

    Parameters
    ----------
    butler: lsst.daf.persistence.Butler
        Helper object providing access to a data repository
    ra: float
        Right ascension of the center of the cutout, in degrees
    dec: float
        Declination of the center of the cutout, in degrees
    visit: int
        Visit id of the calexp's visit
    detector: int
        Detector for the calexp
    cutoutSideLength: float [optional]
        Size of the cutout region in pixels.

    Returns
    -------
    MaskedImage
    """
    
    dataId = {'visit': visit, 'detector': detector}    
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)    
    calexp_wcs = butler.get('calexp.wcs', **dataId)
    xy = geom.PointI(calexp_wcs.skyToPixel(radec))
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    parameters = {'bbox': bbox}
    cutout_image = butler.get('calexp', parameters=parameters, **dataId)

    return cutout_image

In [ ]:
def create_rgb(image, bgr="gri", stretch=1, Q=10, scale=None):
    """
    Create an RGB color composite image.

    Parameters
    ----------
    image : `MultibandExposure`
        `MultibandExposure` to display.
    bgr : sequence
        A 3-element sequence of filter names (i.e., keys of the exps dict)
        indicating what band to use for each channel. If `image` only has
        three filters then this parameter is ignored and the filters
        in the image are used.
    stretch: int
        The linear stretch of the image.
    Q: int
        The Asinh softening parameter.
    scale: list of 3 floats, each less than 1. (default: None)
        Re-scales the RGB channels.

    Returns
    -------
    rgb: ndarray
        RGB (integer, 8-bits per channel) colour image as an NxNx3 numpy array.
    """

    # If the image only has 3 bands, reverse the order of the bands
    #   to produce the RGB image
    if len(image) == 3:
        bgr = image.filters

    # Extract the primary image component of each Exposure with the
    #   .image property, and use .array to get a NumPy array view.

    if scale is None:
        r_im = image[bgr[2]].array  # numpy array for the r channel
        g_im = image[bgr[1]].array  # numpy array for the g channel
        b_im = image[bgr[0]].array  # numpy array for the b channel
    else:
        # manually re-scaling the images here
        r_im = image[bgr[2]].array * scale[0]
        g_im = image[bgr[1]].array * scale[1]
        b_im = image[bgr[0]].array * scale[2]

    rgb = make_lupton_rgb(image_r=r_im,
                          image_g=g_im,
                          image_b=b_im,
                          stretch=stretch, Q=Q)
    # "stretch" and "Q" are parameters to stretch and scale the pixel values

    return rgb

In [ ]:
def find_neighbourg(ra0,dec0,src):
    """ 
    """
    all_angles = []
    for index, row in src.iterrows():
        spherePointCenter = lsst.geom.SpherePoint(ra0*lsst.geom.degrees, dec0*lsst.geom.degrees)
        spt = lsst.geom.SpherePoint(row["ra"]*lsst.geom.degrees, row["dec"]*lsst.geom.degrees)
        ang = spherePointCenter.separation(spt)
        ang_arcsec = ang.asArcseconds()
        all_angles.append(ang_arcsec)
    return all_angles

In [ ]:
def PlotAndFitHisto(data,std_clip,ax,nbins=50,xlabel=r'$(G-R)_1 - (G-R)_2$ :',title="Differential color with 2 stars",xmin=-100.,xmax=100.):
    #data = df_col_gr["ratio3"].values

    hist, bin_edges = np.histogram(data,bins=nbins,range=(xmin,xmax),density=True)
    bin_centres = (bin_edges[:-1] + bin_edges[1:])/2

    # Define model function to be used to fit to the data above:
    def gauss(x, *p):
        A, mu, sigma = p
        return A*np.exp(-(x-mu)**2/(2.*sigma**2))

    # p0 is the initial guess for the fitting coefficients (A, mu and sigma above)
    p0 = [1., 0., 1.]

    coeff, var_matrix = curve_fit(gauss, bin_centres, hist, p0=p0)

    # Get the fitted curve
    hist_fit = gauss(bin_centres, *coeff)

    textstr = '\n'.join((
    xlabel,
    r'$\sigma_{clipped} = %.01f$ mmag' %(std_clip,),
    r'$\sigma_{fit} = %.01f$ mmag' %(coeff[2], )))

    ax.step(bin_centres, hist,'k' ,label='data')
    ax.plot(bin_centres, hist_fit,'r-', label='fit')
    ax.set_xlabel(xlabel)
    ax.legend()
    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=16,verticalalignment='top',color="blue", weight='bold',bbox=props)
    ax.set_title(title)
    ax.set_xlim(xmin,xmax)
    # Finally, lets get the fitting parameters, i.e. the mean and standard deviation:
    #print('Fitted mean = ', coeff[1])
    #print('Fitted standard deviation = ', coeff[2])

## Get Pixel Scale

In [ ]:
import lsst.geom as geom
import lsst.sphgeom

skymap = butler.get('skyMap', skymap=skymapName, collections=collections )
tractInfo = skymap.generateTract(tract)
for patch in tractInfo:    
    patchID = patch.getSequentialIndex()
        
    ibb=patch.getInnerBBox()
    tWCS=tractInfo.getWcs()
       
    # loop on the 4 corners
    for icorn,corner in enumerate(ibb.getCorners()):
        p = geom.Point2D(corner.getX(), corner.getY())
        coord = tWCS.pixelToSky(p)

In [ ]:
tWCS

In [ ]:
#arcsec/pixel
pixel_scale = tWCS.getPixelScale().asArcseconds()

## Selected visits

In [ ]:
#inputfilename = "sources_objectTable-t3864-multibands-o912-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240414_w_2024_15_PREOPS-5069.csv"
inputfilename1 = "sources_objectTable-t3864-multibands-o617-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240414_w_2024_15_PREOPS-5069.csv"
inputfilename2= "sources_objectTable-t3864-multibands-o647-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240414_w_2024_15_PREOPS-5069.csv"
df_myselectedvisits1 = pd.read_csv(inputfilename1 ,index_col=0)
df_myselectedvisits2 = pd.read_csv(inputfilename2 ,index_col=0)
tract = 3864
#objectname = 547
#objectname = 912
objectname1 = 617
objectname2 = 647
path = f"calexp_t{tract}_allband_o{objectname1}_{objectname2}"
title = f"Auxtel Light Curves : tract = {tract}, objectIdx = {objectname1}, {objectname2} "
suptitle = inputfilename1 

In [ ]:
if not os.path.exists(path):
    os.mkdir(path)

In [ ]:
df_myselectedvisits1.sort_values("visit",inplace=True)
df_myselectedvisits2.sort_values("visit",inplace=True)
#df_myselectedvisits.sort_index(inplace=True)

In [ ]:
df_myselectedvisits1.head()

In [ ]:
df_myselectedvisits2.head()

In [ ]:
df_myselectedvisits1["nightObs"] = df_myselectedvisits1.apply(lambda x: x['visit']//100_000 , axis=1)
df_myselectedvisits1["Time"] = pd.to_datetime(df_myselectedvisits1['obsStart'])

df_myselectedvisits2["nightObs"] = df_myselectedvisits2.apply(lambda x: x['visit']//100_000 , axis=1)
df_myselectedvisits2["Time"] = pd.to_datetime(df_myselectedvisits2['obsStart'])

In [ ]:
df_myselectedvisits_g1 = df_myselectedvisits1[df_myselectedvisits1.band == "g"]
df_myselectedvisits_r1 = df_myselectedvisits1[df_myselectedvisits1.band == "r"]
df_myselectedvisits_z1 = df_myselectedvisits1[df_myselectedvisits1.band == "z"]
df_myselectedvisits_y1 = df_myselectedvisits1[df_myselectedvisits1.band == "y"]

df_myselectedvisits_g2 = df_myselectedvisits2[df_myselectedvisits2.band == "g"]
df_myselectedvisits_r2 = df_myselectedvisits2[df_myselectedvisits2.band == "r"]
df_myselectedvisits_z2 = df_myselectedvisits2[df_myselectedvisits2.band == "z"]
df_myselectedvisits_y2 = df_myselectedvisits2[df_myselectedvisits2.band == "y"]

In [ ]:
# no 2023112100507 in z
df_myselectedvisits_y1[["nightObs","visit"]].head() 

In [ ]:
df_myselectedvisits_y2[["nightObs","visit"]].head() 

In [ ]:
df_y = df_myselectedvisits_y1.merge(df_myselectedvisits_y2, left_on='visit', right_on='visit',how="inner")
df_z = df_myselectedvisits_z1.merge(df_myselectedvisits_z2, left_on='visit', right_on='visit',how="inner")
df_g = df_myselectedvisits_g1.merge(df_myselectedvisits_g2, left_on='visit', right_on='visit',how="inner")
df_r = df_myselectedvisits_r1.merge(df_myselectedvisits_r2, left_on='visit', right_on='visit',how="inner")

In [ ]:
df_y.head()

### Distance in pixels

In [ ]:
df_y["dx"] = df_y['x_x'] - df_y['x_y']
df_y["dy"] = df_y['y_x'] - df_y['y_y']
df_y["dpix"] = np.sqrt(df_y["dx"]**2 + df_y["dy"]**2)

In [ ]:
df_y["dpix"].hist(bins=50) 

In [ ]:
df_y["expMidptMJD"] = df_y["expMidptMJD_x"]
df_z["expMidptMJD"] = df_z["expMidptMJD_x"]
df_g["expMidptMJD"] = df_g["expMidptMJD_x"]
df_r["expMidptMJD"] = df_r["expMidptMJD_x"]

In [ ]:
df_y["dpsfMag"] = df_y["psfMag_x"] -  df_y["psfMag_y"]
df_z["dpsfMag"] = df_z["psfMag_x"] -  df_z["psfMag_y"]
df_g["dpsfMag"] = df_g["psfMag_x"] -  df_g["psfMag_y"]
df_r["dpsfMag"] = df_r["psfMag_x"] -  df_r["psfMag_y"]

In [ ]:
df_y["dpsfMagErr"] = np.sqrt(df_y["psfMagErr_x"]**2 +  df_y["psfMagErr_y"]**2) 
df_z["dpsfMagErr"] = np.sqrt(df_z["psfMagErr_x"]**2 +  df_z["psfMagErr_y"]**2) 
df_g["dpsfMagErr"] = np.sqrt(df_g["psfMagErr_x"]**2 +  df_g["psfMagErr_y"]**2) 
df_r["dpsfMagErr"] = np.sqrt(df_r["psfMagErr_x"]**2 +  df_r["psfMagErr_y"]**2) 

In [ ]:
df_y["dapFlux_35_0_calMag"] = df_y["apFlux_35_0_calMag_x"] - df_y["apFlux_35_0_calMag_y"]
df_z["dapFlux_35_0_calMag"] = df_z["apFlux_35_0_calMag_x"] - df_z["apFlux_35_0_calMag_y"]
df_g["dapFlux_35_0_calMag"] = df_g["apFlux_35_0_calMag_x"] - df_g["apFlux_35_0_calMag_y"]
df_r["dapFlux_35_0_calMag"] = df_r["apFlux_35_0_calMag_x"] - df_r["apFlux_35_0_calMag_y"]

In [ ]:
df_y["dapFlux_35_0_calMagErr"] = np.sqrt(df_y["apFlux_35_0_calMagErr_x"]**2 + df_y["apFlux_35_0_calMagErr_y"]**2)
df_z["dapFlux_35_0_calMagErr"] = np.sqrt(df_z["apFlux_35_0_calMagErr_x"]**2 + df_z["apFlux_35_0_calMagErr_y"]**2)
df_g["dapFlux_35_0_calMagErr"] = np.sqrt(df_g["apFlux_35_0_calMagErr_x"]**2 + df_g["apFlux_35_0_calMagErr_y"]**2)
df_r["dapFlux_35_0_calMagErr"] = np.sqrt(df_r["apFlux_35_0_calMagErr_x"]**2 + df_r["apFlux_35_0_calMagErr_y"]**2)

In [ ]:
df_y["d_psfMag"] = df_y["dpsfMag"] - df_y["dpsfMag"].mean()
df_z["d_psfMag"] = df_z["dpsfMag"] - df_z["dpsfMag"].mean()
df_g["d_psfMag"] = df_g["dpsfMag"] - df_g["dpsfMag"].mean()
df_r["d_psfMag"] = df_r["dpsfMag"] - df_r["dpsfMag"].mean()

In [ ]:

df_y["d_apFlux_35_0_calMag"] = df_y["dapFlux_35_0_calMag"] - df_y["dapFlux_35_0_calMag"].mean()
df_z["d_apFlux_35_0_calMag"] = df_z["dapFlux_35_0_calMag"] - df_z["dapFlux_35_0_calMag"].mean()
df_g["d_apFlux_35_0_calMag"] = df_g["dapFlux_35_0_calMag"] - df_g["dapFlux_35_0_calMag"].mean()
df_r["d_apFlux_35_0_calMag"] = df_r["dapFlux_35_0_calMag"] - df_r["dapFlux_35_0_calMag"].mean()

In [ ]:
fig,(ax1,ax2,ax3,ax4) = plt.subplots(4,1,figsize=(12,10),sharex=True,sharey=True)
title = f"AuxtelLightCurves multi-band tract = {tract} objidx = {objectname1}-{objectname2}"
ax1.errorbar(df_g.expMidptMJD,df_g.d_psfMag,yerr=df_g.dpsfMagErr,fmt="o",color="g",label="band g")
ax1.set_title(title)
ax2.errorbar(df_r.expMidptMJD,df_r.d_psfMag,yerr=df_r.dpsfMagErr,fmt="o",color="r",label="band r")
ax3.errorbar(df_z.expMidptMJD,df_z.d_psfMag,yerr=df_z.dpsfMagErr,fmt="o",color="grey",label="band z")
ax4.errorbar(df_y.expMidptMJD,df_y.d_psfMag,yerr=df_y.dpsfMagErr,fmt="o",color="k",label="band y")
#df_r.plot.scatter(x="expMidptMJD",y="d_psfMag",ax=ax2,s=20,c="zeroPoint_x",cmap="jet",label="band r",grid=True,rot=45)
#df_z.plot.scatter(x="expMidptMJD",y="d_psfMag",ax=ax3,s=20,c="zeroPoint_x",cmap="jet",label="band z",grid=True,rot=45)
#df_y.plot.scatter(x="expMidptMJD",y="d_psfMag",ax=ax4,s=20,c="zeroPoint_x",cmap="jet",label="band y",grid=True,rot=45)
#plt.setp(ax1.get_xticklabels(), visible=False)
#plt.setp(ax2.get_xticklabels(), visible=False)
#plt.setp(ax3.get_xticklabels(), visible=False)
#plt.setp(ax4.get_xticklabels(), visible=True)
#ax4.tick_params(labelbottom=True)
ax1.set_ylim(-0.025,0.025)
ax1.grid()
ax2.grid()
ax3.grid()
ax4.grid()
ax1.legend()
ax2.legend()
ax3.legend()
ax4.legend()
ax4.set_xlabel("expMidptMJD")
ax1.set_ylabel("$\Delta \; psfMag$ (mag)")
ax2.set_ylabel("$\Delta \; psfMag$ (mag)")
ax3.set_ylabel("$\Delta \; psfMag$ (mag)")
ax4.set_ylabel("$\Delta \; psfMag$ (mag)")
plt.suptitle(suptitle)
plt.tight_layout()
plt.show()

In [ ]:
fig,(ax1,ax2,ax3,ax4) = plt.subplots(4,1,figsize=(12,10),sharex=True,sharey=True)
title = f"AuxtelLightCurves multi-band tract = {tract} objidx = {objectname1}-{objectname2}"
df_g.plot.scatter(x="expMidptMJD",y="d_psfMag",ax=ax1,s=20,c="zeroPoint_x",cmap="jet",label="band g",grid=True,rot=45)
ax1.set_title(title)
df_r.plot.scatter(x="expMidptMJD",y="d_psfMag",ax=ax2,s=20,c="zeroPoint_x",cmap="jet",label="band r",grid=True,rot=45)
df_z.plot.scatter(x="expMidptMJD",y="d_psfMag",ax=ax3,s=20,c="zeroPoint_x",cmap="jet",label="band z",grid=True,rot=45)
df_y.plot.scatter(x="expMidptMJD",y="d_psfMag",ax=ax4,s=20,c="zeroPoint_x",cmap="jet",label="band y",grid=True,rot=45)
#plt.setp(ax1.get_xticklabels(), visible=False)
#plt.setp(ax2.get_xticklabels(), visible=False)
#plt.setp(ax3.get_xticklabels(), visible=False)
#plt.setp(ax4.get_xticklabels(), visible=True)
ax4.tick_params(labelbottom=True)
ax4.set_xlabel("expMidptMJD")
ax1.set_ylim(-0.025,0.025)
plt.suptitle(suptitle)
plt.tight_layout()
plt.show()

In [ ]:
fig,(ax1,ax2,ax3,ax4) = plt.subplots(4,1,figsize=(12,10),sharex=True,sharey=True)
title = f"AuxtelLightCurves multi-band tract = {tract} objidx = {objectname1}-{objectname2}"
ax1.errorbar(df_g.expMidptMJD,df_g.d_apFlux_35_0_calMag,yerr=df_g.dapFlux_35_0_calMagErr,fmt="o",color="g",label="band g")
ax1.set_title(title)
ax2.errorbar(df_r.expMidptMJD,df_r.d_apFlux_35_0_calMag,yerr=df_r.dapFlux_35_0_calMagErr,fmt="o",color="r",label="band r")
ax3.errorbar(df_z.expMidptMJD,df_z.d_apFlux_35_0_calMag,yerr=df_z.dapFlux_35_0_calMagErr,fmt="o",color="grey",label="band z")
ax4.errorbar(df_y.expMidptMJD,df_y.d_apFlux_35_0_calMag,yerr=df_y.dapFlux_35_0_calMagErr,fmt="o",color="k",label="band y")
#df_r.plot.scatter(x="expMidptMJD",y="d_psfMag",ax=ax2,s=20,c="zeroPoint_x",cmap="jet",label="band r",grid=True,rot=45)
#df_z.plot.scatter(x="expMidptMJD",y="d_psfMag",ax=ax3,s=20,c="zeroPoint_x",cmap="jet",label="band z",grid=True,rot=45)
#df_y.plot.scatter(x="expMidptMJD",y="d_psfMag",ax=ax4,s=20,c="zeroPoint_x",cmap="jet",label="band y",grid=True,rot=45)
#plt.setp(ax1.get_xticklabels(), visible=False)
#plt.setp(ax2.get_xticklabels(), visible=False)
#plt.setp(ax3.get_xticklabels(), visible=False)
#plt.setp(ax4.get_xticklabels(), visible=True)
#ax4.tick_params(labelbottom=True)
ax1.set_ylim(-0.025,0.025)
ax1.grid()
ax2.grid()
ax3.grid()
ax4.grid()
ax1.legend()
ax2.legend()
ax3.legend()
ax4.legend()
ax4.set_xlabel("expMidptMJD")
ax1.set_ylabel("$\Delta \; Ap35 Mag$ (mag)")
ax2.set_ylabel("$\Delta \; Ap35 Mag$ (mag)")
ax3.set_ylabel("$\Delta \; Ap35 Mag$ (mag)")
ax4.set_ylabel("$\Delta \; Ap35 Mag$ (mag)")
plt.suptitle(suptitle)
plt.tight_layout()
plt.show()

In [ ]:
fig,(ax1,ax2,ax3,ax4) = plt.subplots(4,1,figsize=(12,10),sharex=True,sharey=True)
title = f"AuxtelLightCurves multi-band tract = {tract} objidx = {objectname1}, {objectname2}"
df_g.plot.scatter(x="expMidptMJD",y="d_apFlux_35_0_calMag",ax=ax1,s=20,c="zeroPoint_x",cmap="jet",label="band g",grid=True,rot=45)
ax1.set_title(title)
df_r.plot.scatter(x="expMidptMJD",y="d_apFlux_35_0_calMag",ax=ax2,s=20,c="zeroPoint_x",cmap="jet",label="band r",grid=True,rot=45)
df_z.plot.scatter(x="expMidptMJD",y="d_apFlux_35_0_calMag",ax=ax3,s=20,c="zeroPoint_x",cmap="jet",label="band z",grid=True,rot=45)
df_y.plot.scatter(x="expMidptMJD",y="d_apFlux_35_0_calMag",ax=ax4,s=20,c="zeroPoint_x",cmap="jet",label="band y",grid=True,rot=45)
#plt.setp(ax1.get_xticklabels(), visible=False)
#plt.setp(ax2.get_xticklabels(), visible=False)
#plt.setp(ax3.get_xticklabels(), visible=False)
#plt.setp(ax4.get_xticklabels(), visible=True)
ax4.tick_params(labelbottom=True)
plt.suptitle(suptitle)
plt.tight_layout()
plt.show()

## Neighbouring visits

In [ ]:
groups_obs_size1 = df_myselectedvisits1.groupby(by=["nightObs","band"]).size()

In [ ]:
groups_obs_size1

In [ ]:
df_groups_obs_size1 = groups_obs_size1.unstack()
df_groups_obs_size1

In [ ]:
#groups_obs_visit = df_myselectedvisits.groupby(by=["nightObs","band","visit"]).mean()["psfMag"]
#groups_obs_visit 

In [ ]:
groups_obs_visit1 = df_myselectedvisits1.groupby(by=["nightObs","band","visit"]).agg(time = ("Time","mean") ,psfMean = ("psfMag","mean"),apFlux_50_0_instFlux = ("apFlux_50_0_instFlux","mean"))
groups_obs_visit1 

In [ ]:
#groups_obs_visit.index

In [ ]:
#groups_obs_visit.query('"nightObs" == 20231121  & "band" == "z"')

In [ ]:
#groups_obs_visit.loc[(20231121,'z',2023112100516)] 

In [ ]:
groups_obs_bandg_visit1 = df_myselectedvisits_g1.groupby(["nightObs","visit"]).agg(time = ("Time","mean"))
groups_obs_bandr_visit1 = df_myselectedvisits_r1.groupby(["nightObs","visit"]).agg(time = ("Time","mean"))
groups_obs_bandz_visit1 = df_myselectedvisits_z1.groupby(["nightObs","visit"]).agg(time = ("Time","mean"))
groups_obs_bandy_visit1 = df_myselectedvisits_y1.groupby(["nightObs","visit"]).agg(time = ("Time","mean"))

In [ ]:
groups_obs_bandg_visit2 = df_myselectedvisits_g2.groupby(["nightObs","visit"]).agg(time = ("Time","mean"))
groups_obs_bandr_visit2 = df_myselectedvisits_r2.groupby(["nightObs","visit"]).agg(time = ("Time","mean"))
groups_obs_bandz_visit2 = df_myselectedvisits_z2.groupby(["nightObs","visit"]).agg(time = ("Time","mean"))
groups_obs_bandy_visit2 = df_myselectedvisits_y2.groupby(["nightObs","visit"]).agg(time = ("Time","mean"))

In [ ]:
groups_obs_bandz_visit1.iloc[:5]

In [ ]:
groups_obs_bandz_visit2.iloc[:5]

## Process night-per night, band by band

We want to associate pairs of observation in different bands:

- visits in g with visits in r,z,y
- visits in r with visits in z,y
- visits in z with visits in y

In [ ]:
def closestvisit(df1,df2):
    """
    This function associate pairs of closest observation

    input : the 2 dataframes (first to one for which one want to find the closest obs in dtaframe 2)
    output : the dataframe of similar format as the first one having columns on nightobs , visit time, dt from second dataframe)
    """
    d1 =df1.copy(deep=True)
    d2 =df2.copy(deep=True)
    all_samples = []
    df = pd.DataFrame(index=d1.index,columns=["nightObs2","visit2","time2","deltat"])
    for key in d1.index:
        time1 = d1.loc[key]["time"]
        d2["dt"] = np.abs(d2["time"]-time1)
        d2["Dt"] = d2["time"]-time1
        cut_sel = d2["dt"] == d2["dt"].min()
        sample = d2[cut_sel]
        #df.iloc[key] = [sample.index[0][0],sample.index[0][1],sample["time"].values[0],sample['dt'].values[0]]
        df.loc[key]["nightObs2"] = sample.index[0][0]
        df.loc[key]["visit2"] = sample.index[0][1]
        df.loc[key]["time2"] = sample["time"].values[0]
        df.loc[key]["deltat"] = pd.to_timedelta(sample["Dt"].values[0], unit="s").total_seconds()
        all_samples.append(sample)
    return df

### process band g and r

In [ ]:
dfg_r = closestvisit(groups_obs_bandg_visit,groups_obs_bandr_visit)
#dfg_r.drop(labels="deltat", axis=1,inplace=True)
dfg_r.columns = ["nightObs_r","visit_r","time_r","dtsec_r"]

In [ ]:
groups_obs_bandg_visit.join(dfg_r).head()

### process band g and z

In [ ]:
dfg_z = closestvisit(groups_obs_bandg_visit,groups_obs_bandz_visit)
#dfg_z.drop(labels="deltat", axis=1,inplace=True)
dfg_z.columns = ["nightObs_z","visit_z","time_z","dtsec_z"]

In [ ]:
dfg_z.head()

### process band g and y

In [ ]:
dfg_y = closestvisit(groups_obs_bandg_visit,groups_obs_bandy_visit)
#dfg_y.drop(labels="deltat", axis=1,inplace=True)
dfg_y.columns = ["nightObs_y","visit_y","time_y","dtsec_y"]

In [ ]:
dfg_y.head()

### Concatenation g with r,z,y

In [ ]:
dfg_rzy = pd.concat([groups_obs_bandg_visit,dfg_r,dfg_z,dfg_y],axis=1)

In [ ]:
dfg_rzy.head() 

### process band r and z

In [ ]:
dfr_z = closestvisit(groups_obs_bandr_visit,groups_obs_bandz_visit)
#dfr_z.drop(labels="deltat", axis=1,inplace=True)
dfr_z.columns = ["nightObs_z","visit_z","time_z","dtsec_z"]

In [ ]:
dfr_z

### process band r and y

In [ ]:
dfr_y = closestvisit(groups_obs_bandr_visit,groups_obs_bandy_visit)
#dfr_y.drop(labels="deltat", axis=1,inplace=True)
dfr_y.columns = ["nightObs_y","visit_y","time_y","dtsec_y"]

In [ ]:
dfr_y

### Concatenation of r with z,y

In [ ]:
dfr_zy = pd.concat([groups_obs_bandr_visit,dfr_z,dfr_y],axis=1)

In [ ]:
dfr_zy.head()

### process band z and y

In [ ]:
dfz_y = closestvisit(groups_obs_bandz_visit,groups_obs_bandy_visit)
#dfz_y.drop(labels="deltat", axis=1,inplace=True)
dfz_y.columns = ["nightObs_y","visit_y","time_y","dtsec_y"]

In [ ]:
dfz_y

### Concatenate z with y

In [ ]:
dfz_y = pd.concat([groups_obs_bandz_visit,dfz_y],axis=1)

In [ ]:
dfz_y

## Fetch a pair of  stars

In [ ]:
def getcutoutandimage(visitId,df_selectedvisits,objectname=912):
    """
    visitId : the calexp where are the two stars
    df_selectedvisits : the list of all sources and visit of the main targeted star
    """
    if objectname == 912:
        boxSize = 500
    else:
        boxSize = 300

    row_source = df_selectedvisits[df_selectedvisits.visit == visitId].iloc[0]  
    band = row_source['band']

    dataId = {'visit': visitId, 'instrument':instrument , 'detector': 0}
    calexp = butler.get('calexp', **dataId,collections=collections)   
    src_cat = butler.get('sourceTable',**dataId, collections=collections)

    #select on star
    src_cat = src_cat[(src_cat["extendedness"]==0) & (src_cat["parentSourceId"] == 0) ]
    #src_cat = src_cat[src_cat["extendedness"]==0 ]

    #select on magnitude : search bright stars
    src_cat["psfMag"] = src_cat["psfFlux"].apply(convert_fluxtomag)
    if objectname == 545:
        src_cat = src_cat[src_cat["psfMag"]<16]
    elif objectname == 912 and band == 'y':
        src_cat = src_cat[src_cat["psfMag"]<15]
    elif objectname == 912 and band == 'g':
        src_cat = src_cat[src_cat["psfMag"]<16]
    elif objectname == 912 and band == 'z':
        src_cat = src_cat[src_cat["psfMag"]<16]
    elif objectname == 912 and band == 'r':
        src_cat = src_cat[src_cat["psfMag"]<16]

 
    x_target = row_source['x']
    y_target = row_source['y']
    ra_target= row_source['ra']
    dec_target= row_source['dec']
    psfMag  = row_source['psfMag']  
    psfSigma = row_source['psfSigma']
    psfMagDiffMmag = row_source['psfMagDiffMmag'] 
    apFlux_50_0_instFlux = row_source['apFlux_50_0_instFlux']
    apFlux_50_0_instFluxErr = row_source['apFlux_50_0_instFluxErr']


    # searcch the neighbouring stars
    separation = find_neighbourg(ra_target,dec_target,src_cat)
    src_cat["sep"] = separation

    #select the source from separation
    if objectname == 547:
        cut_sep = ((src_cat["sep"] > 25) & (src_cat["sep"] < 27)) | (src_cat["sep"] < 1)
        #cut_sep = ( (src_cat["sep"] < 27)) | (src_cat["sep"] < 1)
    elif objectname == 912:
        cut_sep = (src_cat["sep"] < 46)
    src_cat = src_cat[cut_sep]
    src_cat["psfSigma"] = psfSigma
    
    neighbourgs = src_cat.sort_values(by="sep").iloc[:5]
    #print(f"(target :: {x_target:.2f},{y_target:.2f}) , psfMag = {psfMag:.2f} mag, apFlux_50_0_instFlux={apFlux_50_0_instFlux:.0f} ADU" )
    #print(neighbourgs[["x","y","sep","psfMag","apFlux_50_0_instFlux","ra","dec","deblend_skipped"]])

    flag_deblend_skipped = neighbourgs.iloc[0]["deblend_skipped"]

    xSrc = x_target
    ySrc = y_target
    
    minBbox = geom.Point2I(int(xSrc) - boxSize ,int(ySrc) - boxSize)
    maxBbox = geom.Point2I(int(xSrc) + boxSize, int(ySrc) + boxSize)
    srcBbox = geom.Box2I(minBbox, maxBbox)
    # Make the cutout
    try:
        subimg = afwImage.ExposureF(calexp, srcBbox, afwImage.PARENT, True)
        #subimg2 = calexp.getImage().array[int(ySrc) - boxSize: int(ySrc) + boxSize+1, int(xSrc) - boxSize:int(xSrc) + boxSize+1 ]
        #subimgRgb = afwRgb.makeRGB(subimg.image.array, None, None, expMin, expMax - expMin, Q)
        #all_cutout.append(subimgRgb)
        cutout=subimg.image.array
        
    except Exception as inst:
        print(type(inst))    # the exception type
        print(inst.args)     # arguments stored in .args
        print(inst)     
        cutout = None
        
    return neighbourgs,cutout



In [ ]:
def plotfullcalexp(df_pair,ax=None):
    """
    """
    flag_ax = False
    
    if ax is None:
        fig, ax = plt.subplots(1,1,figsize=(8,8))
    else:
        flag_ax = True

    band = 0
    ra = 0
    dec = 0
    
    if len(df_pair)>0:
        visitId = df_pair["visit"].iloc[0]
        band = df_pair["band"].iloc[0]
        psfMag1 = df_pair["psfMag"].iloc[0]
        psfSigma = df_pair["psfSigma"].iloc[0]
        x1 = df_pair["x"].iloc[0]
        y1 = df_pair["y"].iloc[0]
        ra1 = df_pair['ra'].iloc[0]
        dec1 = df_pair['dec'].iloc[0]
        flag_deblend_skipped = df_pair['deblend_skipped'].iloc[0]
        ann1 = Annulus((x1,y1),r=5*psfSigma,width=0.1,color="r")
        ax.add_patch(ann1)

    if len(df_pair)>1:
        psfMag2 = df_pair["psfMag"].iloc[1]
        x2 = df_pair["x"].iloc[1]
        y2 = df_pair["y"].iloc[1]
        ra2 = df_pair['ra'].iloc[1]
        dec2 = df_pair['dec'].iloc[1]
        ann2 = Annulus((x2,y2),r=5*psfSigma,width=0.1,color="orange")
        ax.add_patch(ann2)

    dataId = {'visit': visitId, 'instrument':instrument , 'detector': 0}
    calexp = butler.get('calexp', **dataId,collections=collections)
    imgarr = calexp.image.array 
    shape =  imgarr.shape
   
    z = ZScaleInterval()
    z1,z2 = z.get_limits(imgarr)
    im = ax.imshow(imgarr,cmap="grey",vmin=z1, vmax=z2)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad="1%")
    # useless with zscale
    plt.colorbar(im, cax=cax)

    
    title = f"calexp {visitId}, objidx:{objectname}, band :{band}"
    ax.set_title(title)

    textstr = '\n'.join((
    r'$exposure = %.0f$' % (visitId, ),
    r'$psfMag1 = %.02f mag$' %(psfMag1, ), 
    r'$psfMag2 = %.02f mag$' %(psfMag2, ),  
    r'$ra_1 = %f deg $' %(ra1,),
    r'$ra_2 = %f deg $' %(ra2,),  
    r'$dec_1 = %f deg $' %(dec1,),
    r'$dec_2 = %f deg $' %(dec2,)))  
  

    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=16,verticalalignment='top',color="y", weight='bold',bbox=props) 
    #if flag_deblend_skipped:
    #    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=16,verticalalignment='top',color="r", weight='bold',bbox=props) 
    #else:
    #    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=16,verticalalignment='top',color="g", weight='bold',bbox=props) 

    if not flag_ax:
        plt.show()   

In [ ]:
def plotcutout(df_pair,cutout,ax=None):
    """
    """
    flag_ax = False
    
    if ax is None:
        fig, ax = plt.subplots(1,1,figsize=(5,5))
    else:
        flag_ax = True

    if cutout is None:
        print("No Cutout")
        return

    boxSize= cutout.shape[0]//2
    extent = [ -cutout.shape[1]//2, cutout.shape[1]//2, -cutout.shape[0]//2, cutout.shape[0]//2]
    
    z = ZScaleInterval()
    z1,z2 = z.get_limits(cutout)
    im = ax.imshow(cutout,cmap="grey",vmin=z1, vmax=z2,extent = extent)
    #divider = make_axes_locatable(ax)
    #cax = divider.append_axes("right", size="5%", pad="1%")
    # useless with zscale
    #plt.colorbar(im, cax=cax)

  

    band = 0
    ra = 0
    dec = 0
    
    if len(df_pair)>0:
        visitId = df_pair["visit"].iloc[0]
        band = df_pair["band"].iloc[0]
        psfMag1 = df_pair["psfMag"].iloc[0]
        psfSigma = df_pair["psfSigma"].iloc[0]
        x1 = df_pair["x"].iloc[0]
        y1 = df_pair["y"].iloc[0]
        ra = df_pair['ra'].iloc[0]
        dec = df_pair['dec'].iloc[0]
        flag_deblend_skipped = df_pair['deblend_skipped'].iloc[0]
        ann1 = Annulus((0,0),r=5*psfSigma,width=0.1,color="r")
        ax.add_patch(ann1)

    if len(df_pair)>1:
        psfMag2 = df_pair["psfMag"].iloc[1]
        x2 = df_pair["x"].iloc[1]
        y2 = df_pair["y"].iloc[1]
        dx2 = x2-x1
        # origin in top left corner instead if lower lesft corner
        dy2 = -(y2-y1)
        #dx2 =   x2 -  x1 + boxSize
        #dy2 =   y2 -  y1 + boxSize
        ann2 = Annulus((dx2,dy2),r=5*psfSigma,width=0.1,color="orange")
        ax.add_patch(ann2)
    
    title = f"obj:{objectname}, b:{band}, (ra:{ra:.2f}, dec:{dec:.2f})"
    ax.set_title(title)

    textstr = '\n'.join((
    r'$expos = %.0f$' % (visitId, ),
    r'$psfMag1 = %.02f mag$' %(psfMag1, ), 
    r'$psfMag2 = %.02f mag$' %(psfMag2, ),  
    r'$\sigma_{PSF} = %.2f pix$' % (psfSigma , ),
    r'$deb\_skip = %.0f $' % (flag_deblend_skipped  , )))

   
 

    if flag_deblend_skipped:
        ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=16,verticalalignment='top',color="orange", weight='bold',bbox=props) 
    else:
        ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=16,verticalalignment='top',color="yellow", weight='bold',bbox=props) 

    
    if not flag_ax:
        plt.show()   

In [ ]:
df_pair_g,cutout_g = getcutoutandimage(2023111500494,df_myselectedvisits_g)
df_pair_g["psfMag"] = df_pair_g["psfFlux"].apply(convert_fluxtomag)

In [ ]:
df_pair_y,cutout_y = getcutoutandimage(2023112100509,df_myselectedvisits_y)
df_pair_y["psfMag"] = df_pair_y["psfFlux"].apply(convert_fluxtomag)

In [ ]:
NROWS = 1
NCOLS = 2
fig, (ax1,ax2) = plt.subplots(ncols=NCOLS,nrows=NROWS,figsize=(6*NCOLS,6*NROWS))

plotcutout(df_pair_g,cutout_g,ax=ax1)
plotcutout(df_pair_y,cutout_y,ax=ax2)
plt.show()

In [ ]:
df_pair_g[["ra","dec"]]

In [ ]:
df_pair_g[["ra","dec"]].to_html()

In [ ]:
plotfullcalexp(df_pair_g)

## Differential colors study (star 1 - star 2)

### Build color ratio in G and R band

### G and R band colors

In [ ]:
dfg_r = dfg_rzy[["time","nightObs_r","visit_r","time_r","dtsec_r"]] 
dfg_r.head()

#### Select pairs in g,r within 1 minute

In [ ]:
dfg_r_sel = dfg_r[np.abs(dfg_r["dtsec_r"])<60]
dfg_r_sel.head() 

In [ ]:
df_myselectedvisits_g .head()

# Loop over band pairs exposures neraby in time to calculate colors and relative colors of two stars 

In [ ]:
def GetFluxes2StarsTwoBands(band1,band2,dfb1_b2,df_selectedvisits_b1,df_selectedvisits_b2):
    """
    """
    df_col_b1b2 = pd.DataFrame(columns = ["band1","exposure1","time1","psfMag11","psfMag12","band2","exposure2","time2","psfMag21","psfMag22","dt"])
    entrycount = 0
    for idx,key in enumerate(dfb1_b2.index):
        print(idx,key)
        visit1 = key[1]
        visit2 = dfb1_b2.loc[key]["visit_" + band2]
        time1 = dfb1_b2.loc[key]["time"]
        time2 = dfb1_b2.loc[key]["time_" + band2]
        dt = dfb1_b2.loc[key]["dtsec_" + band2]
        print(f"======================= night :: {key[0]} ==> ({band1},{band2}) = ({visit1},{visit2}) =============================" )
        row_target = df_selectedvisits_b1[df_selectedvisits_b1.visit == visit1].iloc[0]
        #print(row_target)
        # the selected sources retrieved from the sources associated to that calexp
        df_pair_b1,cutoutb1 = getcutoutandimage(visit1,df_selectedvisits_b1,objectname=912) 
        df_pair_b2,cutoutb2 = getcutoutandimage(visit2,df_selectedvisits_b2,objectname=912) 
        # compute magnitudes on psfFlux
        df_pair_b1["psfMag"] = df_pair_b1["psfFlux"].apply(convert_fluxtomag)
        df_pair_b2["psfMag"] = df_pair_b2["psfFlux"].apply(convert_fluxtomag)


        if ( (len(df_pair_b1) == 2) and (len(df_pair_b2) == 2)):
            try:
                psfMag11 =  df_pair_b1["psfMag"].iloc[0]
                psfMag12 =  df_pair_b1["psfMag"].iloc[1]
                psfMag21 =  df_pair_b2["psfMag"].iloc[0]
                psfMag22 =  df_pair_b2["psfMag"].iloc[1]
                df_col_b1b2.loc[entrycount] = [band1,visit1,time1,psfMag11,psfMag12,band2,visit2,time2,psfMag21,psfMag22,dt]
                entrycount+=1
            except Exception as inst:
                print(type(inst))    # the exception type
                print(inst.args)     # arguments stored in .args
                print(inst)
                continue
        else:
            print(">>> Bad size in df_pairs :: ")
            print("dataframes size : ",len(df_pair_b1),len(df_pair_b2))
            print(df_pair_b1.iloc[:,0:4])
            print(df_pair_b2.iloc[:,0:4])
            continue

        if idx%10 == 0:
            if cutoutb1 is not None and cutoutb2 is not None:
                NROWS = 1
                NCOLS = 2
                fig, (ax1,ax2) = plt.subplots(ncols=NCOLS,nrows=NROWS,figsize=(6*NCOLS,6*NROWS))
                plotcutout(df_pair_b1,cutoutb1,ax=ax1)
                plotcutout(df_pair_b2,cutoutb2,ax=ax2)
                plt.show()

    return df_col_b1b2


In [ ]:
df_col_gr = GetFluxes2StarsTwoBands("g","r",dfg_r_sel,df_myselectedvisits_g,df_myselectedvisits_r)

In [ ]:
df_col_gr.head()

In [ ]:
df_col_gr["ratio1"] = (df_col_gr["psfMag11"] - df_col_gr["psfMag12"])*1000
df_col_gr["ratio2"] = (df_col_gr["psfMag11"] -  df_col_gr["psfMag21"])*1000
df_col_gr["ratio3"] = (df_col_gr["psfMag11"] - df_col_gr["psfMag12"]) -  (df_col_gr["psfMag21"] - df_col_gr["psfMag22"])*1000
df_col_gr["g1-r1"] = (df_col_gr["psfMag11"] -  df_col_gr["psfMag21"])
df_col_gr["g2-r2"] = (df_col_gr["psfMag12"] -  df_col_gr["psfMag22"])

In [ ]:
clippedMeanMagArray = scipy.stats.sigmaclip(df_col_gr["ratio1"].values, low=3.0, high=3.0).clipped
clippedMeanMag = np.mean(clippedMeanMagArray)
df_col_gr["ratio1"] = df_col_gr["ratio1"] - clippedMeanMag 
clippedStd1 = np.std(clippedMeanMagArray-clippedMeanMag )
clippedMeanMagArray = scipy.stats.sigmaclip(df_col_gr["ratio2"].values, low=3.0, high=3.0).clipped
clippedMeanMag = np.mean(clippedMeanMagArray)
df_col_gr["ratio2"] = df_col_gr["ratio2"] - clippedMeanMag 
clippedStd2 = np.std(clippedMeanMagArray-clippedMeanMag )
clippedMeanMagArray = scipy.stats.sigmaclip(df_col_gr["ratio3"].values, low=3.0, high=3.0).clipped
clippedMeanMag = np.mean(clippedMeanMagArray)
df_col_gr["ratio3"] = df_col_gr["ratio3"] - clippedMeanMag 
clippedStd3 = np.std(clippedMeanMagArray-clippedMeanMag )

In [ ]:
textstr = '\n'.join((
r'$\sigma_{clipped}((G_1 - G_2)  = %.1f mmag $' %(clippedStd1, ),
r'$\sigma_{clipped}((G-R)_1)  = %.01f mmag$' %(clippedStd2, ),
r'$\sigma_{clipped}((G-R)_1 - (G-R)_2) = %.01f mmag$' %(clippedStd3, )))

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%dT%H:%M")
fig,axs = plt.subplots(1,1,figsize=(14,6))
ax1  = axs
df_col_gr.plot(x="time1",y="ratio1",ax=ax1,marker='o',c="r",lw=0.0,grid=True,label="1) $G_1 - G_2\;\; \sigma_{clip}=$"+f"{clippedStd1:.1f} mmag")
df_col_gr.plot(x="time1",y="ratio2",ax=ax1,marker='o',c="g",lw=0.0,grid=True,label="2) $(G-R)_1 \; \;\sigma_{clip}= $"+f"{clippedStd2:.1f} mmag" )
df_col_gr.plot(x="time1",y="ratio3",ax=ax1,marker='o',c="b",lw=0.0,grid=True,label="3) $(G-R)_1-(G-R)_2 \;\;\sigma_{clip}= $"+f"{clippedStd3:.1f} mmag")
ax1.set_ylabel("dispersion in mmag")
ax1.set_xlabel("time")
ax1.xaxis.set_major_formatter(date_form)
ax1.axhline(color="k")
ax1.set_title("relative color dispersion: $1)G_1-G_2, \;\;2) (G-R)_1, \;\;3) (G-R)_1 - (G-R)_2$")
#ax1.set_ylabel("$(G-R)_1 - (G-R)_2$ mmag")
#ax1.text(0.05, 0.95, textstr, transform=ax1.transAxes, fontsize=16,verticalalignment='top',color="red", weight='bold',bbox=props)
ax1.set_ylim(-100.,100)

In [ ]:
fig,axs = plt.subplots(1,3,figsize=(15,5))
ax1,ax2,ax3  = axs

data = df_col_gr["ratio1"].values
PlotAndFitHisto(data,clippedStd1,ax=ax1,nbins=60,xlabel=r'$G_1 - G_2$ mmag',title="Differ mag  G1-G2 with 2 stars")

data = df_col_gr["ratio2"].values
PlotAndFitHisto(data,clippedStd2,ax=ax2,nbins=60,xlabel=r'$(G-R)_1$ mmag',title="Differ color G-R with 1 star")


data = df_col_gr["ratio3"].values
PlotAndFitHisto(data,clippedStd3,ax=ax3,nbins=60,xlabel=r'$(G-R)_1 - (G-R)_2$ mmag',title="Differ color G-R with 2 stars")


plt.tight_layout()
plt.show()

### G - Y bands

In [ ]:
dfg_y = dfg_rzy[["time","nightObs_y","visit_y","time_y","dtsec_y"]] 
dfg_y.head()

#### select close measurement in time

In [ ]:
dfg_y_sel = dfg_y[np.abs(dfg_y["dtsec_y"]<3600)]

In [ ]:
dfg_y_sel.head()

In [ ]:
df_col_gy = GetFluxes2StarsTwoBands("g","y",dfg_y_sel,df_myselectedvisits_g,df_myselectedvisits_y)

In [ ]:
df_col_gy.head()

In [ ]:
df_col_gy["ratio1"] = (df_col_gy["psfMag11"] - df_col_gy["psfMag12"])*1000
df_col_gy["ratio2"] = (df_col_gy["psfMag11"] -  df_col_gy["psfMag21"])*1000
df_col_gy["ratio3"] = (df_col_gy["psfMag11"] - df_col_gy["psfMag12"]) -  (df_col_gy["psfMag21"] - df_col_gy["psfMag22"])*1000
df_col_gy["g1-y1"] = (df_col_gy["psfMag11"] -  df_col_gy["psfMag21"])
df_col_gy["g2-y2"] = (df_col_gy["psfMag12"] -  df_col_gy["psfMag22"])

In [ ]:
clippedMeanMagArray = scipy.stats.sigmaclip(df_col_gy["ratio1"].values, low=3.0, high=3.0).clipped
clippedMeanMag = np.mean(clippedMeanMagArray)
df_col_gy["ratio1"] = df_col_gy["ratio1"] - clippedMeanMag 
clippedStd1 = np.std(clippedMeanMagArray-clippedMeanMag )
clippedMeanMagArray = scipy.stats.sigmaclip(df_col_gy["ratio2"].values, low=3.0, high=3.0).clipped
clippedMeanMag = np.mean(clippedMeanMagArray)
df_col_gy["ratio2"] = df_col_gy["ratio2"] - clippedMeanMag 
clippedStd2 = np.std(clippedMeanMagArray-clippedMeanMag )
clippedMeanMagArray = scipy.stats.sigmaclip(df_col_gy["ratio3"].values, low=3.0, high=3.0).clipped
clippedMeanMag = np.mean(clippedMeanMagArray)
df_col_gy["ratio3"] = df_col_gy["ratio3"] - clippedMeanMag 
clippedStd3 = np.std(clippedMeanMagArray-clippedMeanMag )

In [ ]:
textstr = '\n'.join((
r'$\sigma_{clipped}((G_1 - G_2)  = %.1f mmag $' %(clippedStd1, ),
r'$\sigma_{clipped}((G-Y)_1)  = %.01f mmag$' %(clippedStd2, ),
r'$\sigma_{clipped}((G-Y)_1 - (G-Y)_2) = %.01f mmag$' %(clippedStd3, )))

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%dT%H:%M")
fig,axs = plt.subplots(1,1,figsize=(14,6))
ax1  = axs
df_col_gy.plot(x="time1",y="ratio1",ax=ax1,marker='o',c="r",lw=0.0,grid=True,label="1) $G_1 - G_2\;\; \sigma_{clip}=$"+f"{clippedStd1:.1f} mmag")
df_col_gy.plot(x="time1",y="ratio2",ax=ax1,marker='o',c="g",lw=0.0,grid=True,label="2) $(G-Y)_1 \; \;\sigma_{clip}= $"+f"{clippedStd2:.1f} mmag" )
df_col_gy.plot(x="time1",y="ratio3",ax=ax1,marker='o',c="b",lw=0.0,grid=True,label="3) $(G-Y)_1-(G-Y)_2 \;\;\sigma_{clip}= $"+f"{clippedStd3:.1f} mmag")
ax1.set_ylabel("dispersion in mmag")
ax1.set_xlabel("time")
ax1.xaxis.set_major_formatter(date_form)
ax1.axhline(color="k")
ax1.set_title("relative color dispersion: $1)G_1-G_2, \;\;2) (G-Y)_1, \;\;3) (G-Y)_1 - (G-Y)_2$")
#ax1.set_ylabel("$(G-R)_1 - (G-R)_2$ mmag")
#ax1.text(0.05, 0.95, textstr, transform=ax1.transAxes, fontsize=16,verticalalignment='top',color="red", weight='bold',bbox=props)
ax1.set_ylim(-100.,100)

In [ ]:
fig,axs = plt.subplots(1,3,figsize=(15,5))
ax1,ax2,ax3  = axs

data = df_col_gy["ratio1"].values
PlotAndFitHisto(data,clippedStd1,ax=ax1,nbins=60,xlabel=r'$G_1 - G_2$ mmag',title="Differ mag  G1-G2 with 2 stars")

data = df_col_gy["ratio2"].values
PlotAndFitHisto(data,clippedStd2,ax=ax2,nbins=60,xlabel=r'$(G-Y)_1$ mmag',title="Differ color G-Y with 1 star")


data = df_col_gy["ratio3"].values
PlotAndFitHisto(data,clippedStd3,ax=ax3,nbins=60,xlabel=r'$(G-Y)_1 - (G-Y)_2$ mmag',title="Differ color G-Y with 2 stars")

plt.tight_layout()
plt.show()

### R - Y bands

In [ ]:
dfr_y = dfr_zy[["time","nightObs_y","visit_y","time_y","dtsec_y"]] 
dfr_y.head()

In [ ]:
dfr_y_sel = dfr_y[np.abs(dfr_y["dtsec_y"])<3600]

In [ ]:
dfr_y_sel

In [ ]:
df_col_ry = GetFluxes2StarsTwoBands("r","y",dfr_y_sel,df_myselectedvisits_r,df_myselectedvisits_y)

In [ ]:
df_col_ry.head() 

In [ ]:
df_col_ry = df_col_ry[np.abs(df_col_ry["dt"])<3600]

In [ ]:
df_col_ry["ratio1"] = (df_col_ry["psfMag11"] - df_col_ry["psfMag12"])*1000
df_col_ry["ratio2"] = (df_col_ry["psfMag11"] - df_col_ry["psfMag21"])*1000
df_col_ry["ratio3"] = (df_col_ry["psfMag11"] - df_col_ry["psfMag12"]) -  (df_col_ry["psfMag21"] - df_col_ry["psfMag22"])*1000
df_col_ry["r1-y1"] = (df_col_ry["psfMag11"] -  df_col_ry["psfMag21"])
df_col_ry["r2-y2"] = (df_col_ry["psfMag12"] -  df_col_ry["psfMag22"])

In [ ]:
clippedMeanMagArray = scipy.stats.sigmaclip(df_col_ry["ratio1"].values, low=3.0, high=3.0).clipped
clippedMeanMag = np.mean(clippedMeanMagArray)
df_col_ry["ratio1"] = df_col_ry["ratio1"] - clippedMeanMag 
clippedStd1 = np.std(clippedMeanMagArray-clippedMeanMag )
clippedMeanMagArray = scipy.stats.sigmaclip(df_col_ry["ratio2"].values, low=3.0, high=3.0).clipped
clippedMeanMag = np.mean(clippedMeanMagArray)
df_col_ry["ratio2"] = df_col_ry["ratio2"] - clippedMeanMag 
clippedStd2 = np.std(clippedMeanMagArray-clippedMeanMag )
clippedMeanMagArray = scipy.stats.sigmaclip(df_col_ry["ratio3"].values, low=3.0, high=3.0).clipped
clippedMeanMag = np.mean(clippedMeanMagArray)
df_col_ry["ratio3"] = df_col_ry["ratio3"] - clippedMeanMag 
clippedStd3 = np.std(clippedMeanMagArray-clippedMeanMag )

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%dT%H:%M")
fig,axs = plt.subplots(1,1,figsize=(14,6))
ax1  = axs
df_col_ry.plot(x="time1",y="ratio1",ax=ax1,marker='o',c="r",lw=0.0,grid=True,label="1) $R_1 - R_2\;\; \sigma_{clip}=$"+f"{clippedStd1:.1f} mmag")
df_col_ry.plot(x="time1",y="ratio2",ax=ax1,marker='o',c="g",lw=0.0,grid=True,label="2) $(R-Y)_1 \; \;\sigma_{clip}= $"+f"{clippedStd2:.1f} mmag" )
df_col_ry.plot(x="time1",y="ratio3",ax=ax1,marker='o',c="b",lw=0.0,grid=True,label="3) $(R-Y)_1-(R-Y)_2 \;\;\sigma_{clip}= $"+f"{clippedStd3:.1f} mmag")
ax1.set_ylabel("dispersion in mmag")
ax1.set_xlabel("time")
ax1.xaxis.set_major_formatter(date_form)
ax1.axhline(color="k")
ax1.set_title("relative color dispersion: $1)R_1-R_2, \;\;2) (R-Y)_1, \;\;3) (R-Y)_1 - (R-Y)_2$")
#ax1.set_ylabel("$(G-R)_1 - (G-R)_2$ mmag")
#ax1.text(0.05, 0.95, textstr, transform=ax1.transAxes, fontsize=16,verticalalignment='top',color="red", weight='bold',bbox=props)
ax1.set_ylim(-100.,100)

In [ ]:
fig,axs = plt.subplots(1,3,figsize=(15,5))
ax1,ax2,ax3  = axs

data = df_col_ry["ratio1"].values
PlotAndFitHisto(data,clippedStd1,ax=ax1,nbins=60,xlabel=r'$R_1 - R_2$ mmag',title="Differ mag  R1-R2 with 2 stars")

data = df_col_ry["ratio2"].values
PlotAndFitHisto(data,clippedStd2,ax=ax2,nbins=60,xlabel=r'$(R-Y)_1$ mmag',title="Differ color R-Y with 1 star")


data = df_col_ry["ratio3"].values
PlotAndFitHisto(data,clippedStd3,ax=ax3,nbins=60,xlabel=r'$(R-Y)_1 - (R-Y)_2$ mmag',title="Differ color R-Y with 2 stars")

plt.tight_layout()
plt.show()

### Z - Y bands

In [ ]:
#dfz_y = dfz_y[["time","nightObs_y","visit_y","time_y","dtsec_y"]] 
dfz_y.head()

In [ ]:
dfz_y_sel = dfz_y[np.abs(dfz_y["dtsec_y"])<3600]

In [ ]:
dfz_y_sel

In [ ]:
# Need to drop  2023112100507 	
#dfz_y_sel = dfz_y_sel.drop(labels=(20231121,2023112100507), axis=0)

In [ ]:
df_myselectedvisits_z['visit']

In [ ]:
#df_myselectedvisits_z [df_myselectedvisits_z.visit == 2023112100507]
#df_myselectedvisits_z [df_myselectedvisits_z.visit == 2023112100535]

In [ ]:
df_col_zy = GetFluxes2StarsTwoBands("z","y",dfz_y_sel,df_myselectedvisits_z,df_myselectedvisits_y)

In [ ]:
df_col_zy = df_col_zy[np.abs(df_col_zy["dt"])<3600]

In [ ]:
df_col_zy

In [ ]:
df_col_zy["ratio1"] = (df_col_zy["psfMag11"] - df_col_zy["psfMag12"])*1000
df_col_zy["ratio2"] = (df_col_zy["psfMag11"] - df_col_zy["psfMag21"])*1000
df_col_zy["ratio3"] = (df_col_zy["psfMag11"] - df_col_zy["psfMag12"]) -  (df_col_zy["psfMag21"] - df_col_zy["psfMag22"])*1000
df_col_zy["z1-y1"] = (df_col_zy["psfMag11"] -  df_col_zy["psfMag21"])
df_col_zy["z2-y2"] = (df_col_zy["psfMag12"] -  df_col_zy["psfMag22"])

In [ ]:
clippedMeanMagArray = scipy.stats.sigmaclip(df_col_zy["ratio1"].values, low=3.0, high=3.0).clipped
clippedMeanMag = np.mean(clippedMeanMagArray)
df_col_zy["ratio1"] = df_col_zy["ratio1"] - clippedMeanMag 
clippedStd1 = np.std(clippedMeanMagArray-clippedMeanMag )
clippedMeanMagArray = scipy.stats.sigmaclip(df_col_zy["ratio2"].values, low=3.0, high=3.0).clipped
clippedMeanMag = np.mean(clippedMeanMagArray)
df_col_zy["ratio2"] = df_col_zy["ratio2"] - clippedMeanMag 
clippedStd2 = np.std(clippedMeanMagArray-clippedMeanMag )
clippedMeanMagArray = scipy.stats.sigmaclip(df_col_zy["ratio3"].values, low=3.0, high=3.0).clipped
clippedMeanMag = np.mean(clippedMeanMagArray)
df_col_zy["ratio3"] = df_col_zy["ratio3"] - clippedMeanMag 
clippedStd3 = np.std(clippedMeanMagArray-clippedMeanMag )

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%dT%H:%M")
fig,axs = plt.subplots(1,1,figsize=(14,6))
ax1  = axs
df_col_zy.plot(x="time1",y="ratio1",ax=ax1,marker='o',c="r",lw=0.0,grid=True,label="1) $Z_1 - Z_2\;\; \sigma_{clip}=$"+f"{clippedStd1:.1f} mmag")
df_col_zy.plot(x="time1",y="ratio2",ax=ax1,marker='o',c="g",lw=0.0,grid=True,label="2) $(Z-Y)_1 \; \;\sigma_{clip}= $"+f"{clippedStd2:.1f} mmag" )
df_col_zy.plot(x="time1",y="ratio3",ax=ax1,marker='o',c="b",lw=0.0,grid=True,label="3) $(Z-Y)_1-(Z-Y)_2 \;\;\sigma_{clip}= $"+f"{clippedStd3:.1f} mmag")
ax1.set_ylabel("dispersion in mmag")
ax1.set_xlabel("time")
ax1.xaxis.set_major_formatter(date_form)
ax1.axhline(color="k")
ax1.set_title("relative color dispersion: $1)Z_1-Z_2, \;\;2) (Z-Y)_1, \;\;3) (Z-Y)_1 - (Z-Y)_2$")
#ax1.set_ylabel("$(G-R)_1 - (G-R)_2$ mmag")
#ax1.text(0.05, 0.95, textstr, transform=ax1.transAxes, fontsize=16,verticalalignment='top',color="red", weight='bold',bbox=props)
ax1.set_ylim(-100.,100)

In [ ]:
fig,axs = plt.subplots(1,3,figsize=(15,5))
ax1,ax2,ax3  = axs

data = df_col_zy["ratio1"].values
PlotAndFitHisto(data,clippedStd1,ax=ax1,nbins=60,xlabel=r'$Z_1 - Z_2$ mmag',title="Differ mag  Z1-Z2 with 2 stars")

data = df_col_zy["ratio2"].values
PlotAndFitHisto(data,clippedStd2,ax=ax2,nbins=60,xlabel=r'$(Z-Y)_1$ mmag',title="Differ color Z-Y with 1 star")


data = df_col_zy["ratio3"].values
PlotAndFitHisto(data,clippedStd3,ax=ax3,nbins=60,xlabel=r'$(Z-Y)_1 - (Z-Y)_2$ mmag',title="Differ color Z-Y with 2 stars")

plt.tight_layout()
plt.show()

# Colors

In [ ]:
a = df_col_gr[["g1-r1","g2-r2"]].describe().loc[["mean","std"]]

In [ ]:
b = df_col_gy[["g1-y1","g2-y2"]].describe().loc[["mean","std"]]

In [ ]:
c = df_col_ry[["r1-y1","r2-y2"]].describe().loc[["mean","std"]]

In [ ]:
d = df_col_zy[["z1-y1","z2-y2"]].describe().loc[["mean","std"]]

In [ ]:
df_col_summ = pd.concat([a,b,c,d],axis=1)

In [ ]:
df_col_summ

In [ ]:
df_col_summ.to_html()

In [ ]:
if 0:
    df_col_gr = pd.DataFrame(columns = ["band1","exposure1","time1","psfMag11","psfMag12","band2","exposure2","time2","psfMag21","psfMag22","dt"])
    band1 = "g"
    band2 = "r"
    entrycount = 0
    for idx,key in enumerate(dfg_r_sel.index):
        visit1 = key[1]
        visit2 = dfg_r_sel.loc[key]["visit_r"]
        time1 = dfg_r_sel.loc[key]["time"]
        time2 = dfg_r_sel.loc[key]["time_r"]
        dt = dfg_r_sel.loc[key]["dtsec_r"]
        print(f"======================= night :: {key[0]} ==> (g,r) = ({visit1},{visit2}) =============================" )
        # the target star from the external list of sources for the object
        row_target = df_myselectedvisits_g[df_myselectedvisits_g.visit == visit1].iloc[0]
        #print(row_target)
        # the selected sources retrieved from the sources associated to that calexp
        df_pair_g,cutoutg = getcutoutandimage(visit1,df_myselectedvisits_g,objectname=912) 
        df_pair_r,cutoutr = getcutoutandimage(visit2,df_myselectedvisits_r,objectname=912) 
        # compute magnitudes on psfFlux
        df_pair_g["psfMag"] = df_pair_g["psfFlux"].apply(convert_fluxtomag)
        df_pair_r["psfMag"] = df_pair_r["psfFlux"].apply(convert_fluxtomag)
    
        if ( (len(df_pair_g) == 2) and (len(df_pair_g) == 2)):
            try:
                psfMag11 =  df_pair_g["psfMag"].iloc[0]
                psfMag12 =  df_pair_g["psfMag"].iloc[1]
                psfMag21 =  df_pair_r["psfMag"].iloc[0]
                psfMag22 =  df_pair_r["psfMag"].iloc[1]
                df_col_gr.loc[entrycount] = [band1,visit1,time1,psfMag11,psfMag12,band2,visit2,time2,psfMag21,psfMag22,dt]
                entrycount+=1
            except Exception as inst:
                print(type(inst))    # the exception type
                print(inst.args)     # arguments stored in .args
                print(inst)
                continue
        else:
            print(">>> Bad size in df_pairs :: ")
            print("dataframes size : ",len(df_pair_g),len(df_pair_r))
            print(df_pair_g.iloc[:,0:4])
            print(df_pair_r.iloc[:,0:4])
            continue
    
          
        if idx%10 == 0:
            if cutoutg is not None and cutoutr is not None:
                NROWS = 1
                NCOLS = 2
                fig, (ax1,ax2) = plt.subplots(ncols=NCOLS,nrows=NROWS,figsize=(6*NCOLS,6*NROWS))
                plotcutout(df_pair_g,cutoutg,ax=ax1)
                plotcutout(df_pair_r,cutoutr,ax=ax2)
                plt.show()
        

In [ ]:
df_col_gr.head()

In [ ]:
df_col_gr["ratio"] = (df_col_gr["psfMag11"] - df_col_gr["psfMag12"]) -  (df_col_gr["psfMag21"] - df_col_gr["psfMag22"])*1000

In [ ]:
clippedMeanMagArray = scipy.stats.sigmaclip(df_col_gr["ratio"].values, low=3.0, high=3.0).clipped
clippedMeanMag = np.mean(clippedMeanMagArray)

In [ ]:
df_col_gr["ratio"] = df_col_gr["ratio"] - clippedMeanMag 
clippedStd = np.std(clippedMeanMagArray-clippedMeanMag )

In [ ]:
textstr = '\n'.join((
r'$(G-R)_1 - (G-R)_2$ :' ,
r'$\sigma_{clipped} = %.02f$ mmag' %(clippedStd, )))

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%dT%H:%M")
fig,axs = plt.subplots(1,1,figsize=(18,5))
ax1  = axs
df_col_gr.plot(x="time1",y="ratio",marker='o',c="b",lw=0.0,ax=ax1,grid=True,label="g-r")
ax1.set_ylabel("ratio")
ax1.xaxis.set_major_formatter(date_form)
ax1.axhline(color="k")
ax1.set_title("relative color dispersion $(G-R)_1 - (G-R)_2$")
ax1.set_ylabel("$(G-R)_1 - (G-R)_2$ mmag")
ax1.text(0.05, 0.95, textstr, transform=ax1.transAxes, fontsize=16,verticalalignment='top',color="red", weight='bold',bbox=props)

https://stackoverflow.com/questions/11507028/fit-a-gaussian-function

In [ ]:
data = df_col_gr["ratio"].values

hist, bin_edges = np.histogram(data,bins=50 ,density=True)
bin_centres = (bin_edges[:-1] + bin_edges[1:])/2

# Define model function to be used to fit to the data above:
def gauss(x, *p):
    A, mu, sigma = p
    return A*np.exp(-(x-mu)**2/(2.*sigma**2))

# p0 is the initial guess for the fitting coefficients (A, mu and sigma above)
p0 = [1., 0., 1.]

coeff, var_matrix = curve_fit(gauss, bin_centres, hist, p0=p0)

# Get the fitted curve
hist_fit = gauss(bin_centres, *coeff)

textstr = '\n'.join((
r'$(G-R)_1 - (G-R)_2$ :' ,
r'$\sigma_{clipped} = %.01f$ mmag' %(clippedStd,),
r'$\sigma_{fit} = %.01f$ mmag' %(coeff[2], )))

fig,axs = plt.subplots(1,1,figsize=(5,5))
ax  = axs

ax.step(bin_centres, hist,'k' ,label='data')
ax.plot(bin_centres, hist_fit,'r-', label='fit')
ax.set_xlabel("$(G-R)_1 - (G-R)_2$ mmag")
ax.legend()
ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=16,verticalalignment='top',color="blue", weight='bold',bbox=props)
ax.set_title("Differential color with 2 stars")
# Finally, lets get the fitting parameters, i.e. the mean and standard deviation:
#print('Fitted mean = ', coeff[1])
#print('Fitted standard deviation = ', coeff[2])

In [ ]:
fig,axs = plt.subplots(1,1,figsize=(5,4))
ax1  = axs
df_col_gr["ratio"].hist(bins=50,lw=2,ax=ax1,histtype="step",color="b",grid=True,label="g-r")
ax1.set_xlabel("$(G-R)_1 - (G-R)_2$ mmag")